### Python Study Group week2

### Theme : 使用偽裝 user-agent 爬取網站

------------------------------------------------------------

### Let's get it started !

```
在對方的網站伺服器沒有設定任何檢查、防爬裝置時，
我們可直接使用 requests 發送請求，對方就會將網站的內容回應給我們。
```

```
但如果對方的伺服器有設定防爬，則我們很有可能會被:
- 拒絕訪問
- 導向其他網站

```

> 此篇主要以被導向其他網站時，該如何解決為目標。

------------------------------------------------------------

### Import Libraries

In [1]:
import requests
from bs4 import BeautifulSoup

from tqdm import tqdm_notebook
import numpy as np
import pandas as pd
import os

------------------------------------------------------------

### 以蝦皮購物網頁為例

In [2]:
def format_url(keyword):
    shopee_search_url = 'https://shopee.tw/search?keyword='
    return "{0}{1}".format(shopee_search_url, keyword)

In [4]:
def get_web_page(url):
    resp = requests.get(url)
    resp.encoding = 'utf-8'
    
    if resp.status_code != 200:
        print('Invalid url:', resp.url)
        return None
    else:
        return resp, resp.text

In [5]:
keyword = 'iphone11'

response, current_page = get_web_page(format_url(keyword))
print("Status code: {0}".format(response.status_code))

Status code: 200


```
由以上我們成功透過 requests 取得網頁回傳，故此時的status code為 200。
```

In [74]:
soup = BeautifulSoup(current_page, 'html.parser')
items = soup.find("div", class_="lazy-image__container mynD6f")
print(items)

None


------------------------------------------------------------

```
過程看似很順遂，但當我要透過 html/css 標籤取得網頁資料時，
伺服器卻不回傳任何資訊給你，發生回傳 None 的狀況。
雖然沒有被重新導向其他網頁，但是user-agent並非是伺服器所認可的user-agent，就會拒絕將商品的資訊回傳給你
只丟給你一個空空的頁面。
```

------------------------------------------------------------

```
接下來，我們將在向伺服器發出 requests 時，傳入 User-Agent 以解決此問題。
```

```
Note:
通常在不用登入之頁面使用 get 去取得網頁資料還會發生問題的，我們可用：
- 發送假 User-Agent
- 偷用 cookies
等方式去解決問題，而這兩種在 requests 模組中使用非常簡單、方便。
```

------------------------------------------------------------

### User-Agent

```bash
用途：
瀏覽器在送出 request 的時候，會送出 header 告訴伺服器(server)一些資訊，
包含瀏覽器是什麼、用什麼主機等···。
而 User-Agent 就是這些 "識別字串"。
```

```bash
首先，先使用 pip 安裝 fake-useragent > ! pip install fake-useragent
接著，我們可初始化 UserAgent 物件，並使用 random 屬性產生隨機的 User-Agent。
```

In [6]:
from fake_useragent import UserAgent
ua = UserAgent()
print(ua.random)

Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/27.0.1500.55 Safari/537.36


------------------------------------------------------------

```
接著，我們將 User-Agent 放入 headers，再將它丟入 requests.get() 當作參數。
```

In [64]:
import requests
from fake_useragent import UserAgent

In [65]:
def get_web_page_2(url, headers):
    resp = requests.get(
        url = url,
        headers = headers  
    )
    if resp.status_code != 200:
        print("Invalid url:", resp.url)
        return None
    else:
        return response, resp.text

In [69]:
keyword = 'iphone11'
iphone11_url = format_url(keyword)
ua = UserAgent()
headers = {
    'User-Agent': ua.random
}

response, current_page = get_web_page_2(iphone11_url, headers)
print("Status code: {0}".format(response.status_code))

Status code: 200


In [95]:
soup = BeautifulSoup(current_page, 'html.parser')
items = soup.find_all("div", class_="_1NoI8_ _2gr36I")
print(items)

[]


------------------------------------------------------------

```bash
同樣地，我們還是無法從伺服器回傳網頁資訊(相同地得到 None)，可能是因為 User-Agent 仍是 Server 所不允許的。
從 Shopee 的 robot.txt 檔，我們可發現哪些 UA 是允許、哪些是 disallow？
```

```
由檔案可發現，當以下幾種狀況是允許的：
- User-Agent: Googlebot
- User-Agent: Bingbot
- Crawl-delay: 0.1
- Crawl-delay: 1
```

In [108]:
keyword = 'iphone11'
iphone11_url = format_url(keyword)
headers = {
    'User-Agent': 'Googlebot',
}

r = requests.get(url,headers=headers)

soup = BeautifulSoup(r.text, 'html.parser')
items = soup.find_all("div", class_="_1NoI8_ _2gr36I")

for idx, item in enumerate(items):
    if idx < 5:
        print(item.text)

iPhone 11 Pro MAX 手機雙攝像頭鏡頭保護圈 蘋果全系iX i11金屬鏡頭保護圈鏡頭加高保護 後置攝像保護
【299免運】iPhone 霧面碳纖維 玻璃貼 保護貼 iPhone11 XR Xs Max i6 i7 i8【D97】
氣囊加厚 iPhone11 pro 手機殼 xs xr i11 i11pro 保護殼 i7plus i8plus 防摔殼
iPhone11 Pro Max 5D滿版XR XS X玻璃貼iPhone8 Plus玻璃保護貼i11 i8 i7 i6
iphone 11 PROMAX XR X 8PLUS i8 i7 防碎邊 5D全玻璃 防指紋 抗油污 高清 滿版保護貼


------------------------------------------------------------

Note:
<br>其實蝦皮有提供 API 讓我們取得網頁商品資訊，它回傳的內容也是熟悉的 Json 形式。